In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 13891, done.
remote: Counting objects: 100% (818/818), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 13891 (delta 500), reused 652 (delta 409), pack-reused 13073
Receiving objects: 100% (13891/13891), 219.95 MiB | 14.31 MiB/s, done.
Resolving deltas: 100% (10114/10114), done.
/content/LLaMA-Factory
assets/       docker-compose.yml  examples/  MANIFEST.in     README_zh.md      setup.py
CITATION.cff  Dockerfile          LICENSE    pyproject.toml  requirements.txt  src/
data/         evaluation/         Makefile   README.md       scripts/          tests/
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.

In [ ]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

In [ ]:
import json

args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",
  dataset="identity,alpaca_en_demo",
  template="llama3",
  finetuning_type="lora",
  lora_target="all",
  output_dir="llama3_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_samples=500,
  max_grad_norm=1.0,
  quantization_bit=4,
  loraplus_lr_ratio=16.0,
  fp16=True,
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

In [ ]:
from llmtuner.chat import ChatModel
from llmtune.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",
  adapter_name_or_path="llama3_lora",
  template="llama3",
  finetuning_type="lora",
  quantization_bit=4,
)
chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()